# WaPOR v3 API Tutorial

In this notebook we'll have a look at how to download WaPOR v3 data using FAO's [GISMGR 2.0 API](https://github.com/un-fao/FAO-Water-Applications/blob/10a70cda0e0023258bc9152e43a73b6e8e20b669/WaPOR/FAO%20GISMGR%202.0%20-%20API%20Reference.pdf) for a specific region (for a notebook on downloading WaPOR v2 data, go [here](https://colab.research.google.com/github/un-fao/FAO-Water-Applications/blob/main/WaPOR/WaPORv2_API.ipynb)). Let's get started by importing the modules we are going to use.

In [15]:
# import ee
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal

import rasterio
import geopandas as gpd
import datetime
import pandas as pd


 the base URL and a function that requests the url and returns some of the information thats returned.

In [16]:
base_url = f"https://data.apps.fao.org/gismgr/api/v2/catalog/workspaces/C3S/mapsets"
def collect_responses(url, info = ["code"]):
    data = {"links": [{"rel": "next", "href": url}]}
    output = list()
    while "next" in [x["rel"] for x in data["links"]]:
        url_ = [x["href"] for x in data["links"] if x["rel"] == "next"][0]
        response = requests.get(url_)
        response.raise_for_status()
        data = response.json()["response"]
        if isinstance(info, list):
            output += [tuple(x.get(y) for y in info) for x in data["items"]]
        else:
            output += data["items"]
    if isinstance(info, list):
        output = sorted(output)
    return output

chose a mapset code to get links to all raster of that mapset

In [18]:
# Read points from shapefile
shapefile = r"../../../GIS/FLUXNET2022.shp"
pts = gpd.read_file(shapefile)
pts = pts = pts[pts['x'].notna()]
coords = [(x,y) for x, y in zip(pts.x, pts.y)]

# Yearly PCP Data

In [19]:
mapset_code = "AGERA5-PF-A"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]
raster_urls

['https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1979.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1980.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1981.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1982.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1983.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1984.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1985.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1986.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-PF-A/C3S.AGERA5-PF-A.1987.tif',
 'https://storage.googleapis

In [20]:
url=raster_urls[0]
url[-8:-4]

'1979'

In [23]:
# Downlaod time series data 
scale = 1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
for url in urls:
    year = int(url[-8:-4])
    dates.append(year)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale


In [25]:
df.to_csv('FLUXNET2022_AGERA5-PF-A_1979-2024.csv')

In [14]:
mapset_code = "L1-AETI-M"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]

In [4]:
# Read points from shapefile
shapefile = r"../0_insitu/Data/FLUXNET2022.shp"
pts = gpd.read_file(shapefile)
pts = pts = pts[pts['x'].notna()]
coords = [(x,y) for x, y in zip(pts.x, pts.y)]

In [5]:
coords

[(-66.7335, -54.9733),
 (-79.9333, 44.3167),
 (-122.9951, 49.119),
 (-122.9849, 49.1293),
 (-80.4123, 43.6405),
 (-122.8414, 55.1119),
 (-73.319, -3.8344),
 (-89.6067, 46.0308),
 (-97.4888, 36.6058),
 (-71.2881, 44.0646),
 (-121.4993, 38.0992),
 (-121.5351, 38.1091),
 (-86.5406, 39.2167),
 (-148.3208, 64.6955),
 (-148.3121, 64.7013),
 (-148.33, 64.6936),
 (-148.3235, 64.6963),
 (-117.0821, 46.7815),
 (-117.0908, 46.784),
 (-117.1261, 46.7551),
 (-117.1285, 46.7518),
 (-89.5379, 44.1031),
 (-89.5002, 44.1467),
 (-89.5727, 44.1394),
 (-89.5475, 44.1597),
 (-89.7117, 43.3448),
 (-121.549, 38.1235),
 (-122.114, 37.6156),
 (-149.2536, 63.8784),
 (-106.2399, 41.3665),
 (-72.1715, 42.5378),
 (-79.1957, 33.3455),
 (-79.244, 33.3242),
 (-79.2322, 33.3482),
 (-119.4641, 46.6889),
 (-119.4614, 46.6878),
 (-68.747, 45.2091),
 (-77.8488, 40.8608),
 (-149.2958, 68.6068),
 (-149.311, 68.6058),
 (-149.3041, 68.6063),
 (-106.635, 32.582),
 (-106.6032, 32.5849),
 (-95.1907, 39.0561),
 (-97.5684, 38.7745

In [16]:
pts

,field_1,Site,Period,Dataset,x,y,geometry
0,0,AR-TF1,2016-2018,AMF,-66.733500,-54.973300,POINT (-66.7335 -54.9733)
1,1,CA-Cbo,1994-2020,AMF,-79.933300,44.316700,POINT (-79.9333 44.3167)
2,2,CA-DB2,2019-2020,AMF,-122.995100,49.119000,POINT (-122.9951 49.119)
3,3,CA-DBB,2014-2020,AMF,-122.984900,49.129300,POINT (-122.9849 49.1293)
4,4,CA-ER1,2015-2020,AMF,-80.412300,43.640500,POINT (-80.4123 43.6405)
...,...,...,...,...,...,...,...
267,267,SE-Deg,2001-2020,ICOS-WW,19.557448,64.182333,POINT (19.55745 64.18233)
268,268,SE-Htm,2015-2020,ICOS-WW,13.417893,56.097566,POINT (13.41789 56.09757)
269,269,SE-Nor,2014-2020,ICOS-WW,17.478520,60.086990,POINT (17.47852 60.08699)
270,270,SE-Ros,2014-2020,ICOS-WW,19.738000,64.172500,POINT (19.738 64.1725)


In [20]:
raster_urls

['https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-01.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-02.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-03.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-04.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-05.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-06.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-07.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-08.tif',
 'https://storage.googleapis.com/fao-gismgr-wapo

In [21]:
# Downlaod time series data 
scale = 0.1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
for url in urls:
    date = pd.to_datetime(url[-11:-4]) 
    dates.append(date)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale


In [24]:
df.to_csv('Data/FLUXNET2022_L1_AETI_M.csv')

In [3]:
mapset_code = "L1-AETI-D"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]
raster_urls

['https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-03-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-03-D2.tif',
 'https://storage.google

In [40]:
# Downlaod time series data 
scale = 0.1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
dekad_day = {
   'D1':'01',
    'D2': '11',
    'D3': '21'    
}

for url in urls:
    date = pd.to_datetime(url[-14:-6]+dekad_day[url[-6:-4]]) 
    dates.append(date)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale
df.to_csv('Data/FLUXNET2022_L1_AETI_D.csv')


In [37]:
df

Site,AR-TF1,CA-Cbo,CA-DB2,CA-DBB,CA-ER1,CA-LP1,PE-QFR,US-ALQ,US-ARM,US-Bar,...,IT-Ren,IT-SR2,IT-Tor,RU-Fy2,RU-Fyo,SE-Deg,SE-Htm,SE-Nor,SE-Ros,SE-Svb
2018-01-01,1.6,0.0,0.1,0.1,0.0,0.0,1.6,0.0,0.0,0.0,...,0.0,0.6,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.7,0.0,0.2,0.1,0.0,0.0,1.6,0.0,0.0,0.0,...,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01,1.8,0.0,0.2,0.2,0.0,0.0,2.1,0.0,0.5,0.0,...,0.1,0.6,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.5,0.0,0.1,0.1,0.0,0.0,2.6,0.0,0.2,0.0,...,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01,1.5,0.0,0.1,0.1,0.0,0.0,2.4,0.0,0.3,0.1,...,0.1,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01,1.1,0.9,0.3,0.2,1.1,0.0,1.9,0.9,1.1,0.5,...,0.4,1.2,0.2,0.0,0.0,0.1,0.2,0.0,0.1,0.1
2018-01-01,1.1,0.5,0.1,0.1,0.5,0.0,2.7,0.3,0.2,0.3,...,0.6,1.3,0.2,0.0,0.0,0.0,0.0,0.0,0.1,0.1
2018-01-01,1.3,0.2,0.2,0.1,0.1,0.0,2.4,0.0,0.5,0.2,...,0.3,1.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.7,0.0,0.1,0.0,0.0,0.0,2.9,0.0,0.3,0.1,...,0.1,0.9,0.0,0.0,0.0,0.0,0.2,0.0,0.1,0.3


# Yearly RET Data

AGERA5-ET0-A

In [26]:
mapset_code = "AGERA5-ET0-A"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]
raster_urls

['https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1979.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1980.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1981.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1982.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1983.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1984.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1985.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1986.tif',
 'https://storage.googleapis.com/fao-gismgr-c3s-data/DATA/C3S/MAPSET/AGERA5-ET0-A/C3S.AGERA5-ET0-A.1987.tif',
 'https://

In [27]:
url=raster_urls[0]
url[-8:-4]

'1979'

In [28]:
# Downlaod time series data 
scale = 1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
for url in urls:
    year = int(url[-8:-4])
    dates.append(year)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale
df

Site,AR-TF1,CA-Cbo,CA-DB2,CA-DBB,CA-ER1,CA-LP1,PE-QFR,US-ALQ,US-ARM,US-Bar,...,IT-Ren,IT-SR2,IT-Tor,RU-Fy2,RU-Fyo,SE-Deg,SE-Htm,SE-Nor,SE-Ros,SE-Svb
1979,465.772766,840.665222,738.491760,738.491760,805.704041,593.921753,1267.446777,728.077393,1447.280396,690.419800,...,616.569092,972.567688,557.067566,582.822449,582.822449,466.254639,570.244873,554.462280,466.882385,460.118378
1980,453.862396,812.434570,656.314026,656.314026,788.452576,574.268433,1277.778320,746.398499,1623.761597,723.848267,...,616.035767,900.263306,547.902100,506.558136,506.558136,497.947174,538.489624,552.647522,499.617859,488.532104
1981,466.135162,811.178467,667.170410,667.170410,793.583313,620.308655,1248.210327,739.903137,1465.388916,720.997620,...,636.843018,945.882751,561.724609,595.619141,595.619141,475.004639,566.525024,581.576355,471.542786,459.661438
1982,487.157471,832.837585,704.403503,704.403503,797.502808,578.010742,1215.795898,730.971497,1395.090210,705.264465,...,646.935608,972.705750,567.030457,533.491211,533.491211,517.326965,613.891235,607.699158,518.332886,507.284058
1983,481.062988,864.744324,692.010132,692.010132,844.745850,562.809204,1275.902100,768.909607,1296.610107,726.217712,...,662.948669,967.674805,570.648743,576.384094,576.384094,507.356354,616.122498,620.281006,504.689972,493.907898
1984,474.669739,826.251465,663.850403,663.850403,809.556580,574.783325,1213.148804,760.964355,1417.788696,702.405518,...,632.968811,944.053162,541.823242,548.026306,548.026306,491.279663,587.390503,546.308228,489.658508,482.499573
1985,470.864990,847.948364,719.670593,719.670593,844.704224,628.601562,1232.413208,750.632141,1338.791504,721.622131,...,653.011292,967.521851,589.046753,528.946045,528.946045,462.228180,559.578918,543.299377,464.104797,450.762939
1986,464.633484,840.384033,690.383301,690.383301,812.195679,596.670959,1186.255737,755.468140,1434.084961,694.579163,...,649.566528,994.987732,583.920532,548.551147,548.551147,479.977509,598.449402,586.595215,478.281036,473.017853
1987,479.052551,912.182190,749.557983,749.557983,897.326538,630.817627,1224.469604,815.871338,1389.092041,730.144531,...,638.401611,973.422791,566.608154,492.043274,492.043274,442.609131,518.492065,507.728058,440.685455,429.239532
1988,476.431976,923.849487,695.935242,695.935242,916.579529,596.443054,1241.787354,862.009705,1433.372803,727.537964,...,651.440674,978.562866,569.979492,575.003479,575.003479,510.074005,595.411865,613.609253,508.901550,498.453979


In [29]:
df.to_csv('FLUXNET2022_AGERA5-ET0-A_1979-2024.csv')